In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import time
from os.path import exists, realpath
from urllib.parse import quote
import json
import glob

current_record = 0
current_set = 0
fails = 0

In [74]:
s = requests.Session()
cookie = 'PHPSESSID=c8hcufru9lt9puq06ct3map455'
def get_case_deets(cino, case_no, court_no, state_code, dist_code):
    
    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="97", "Chromium";v="97"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
        'sec-ch-ua-platform': '"Windows"',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Origin': 'https://services.ecourts.gov.in',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://services.ecourts.gov.in/',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie': cookie
    }

    data = {
      '__csrf_magic': 'sid:22e25a4a3b8a2c3dd3a08f8c43e207354a236ed2,1641670946',
      'state_code': state_code,
      'dist_code': dist_code,
      'cino': cino,
      'case_no': case_no,
      'appFlag': '',
      'court_code': court_no
    }

    response = s.post('https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/o_civil_case_history.php', headers=headers,data=data)
    return (response.content, 0)

def get_case_document(link):

    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="97", "Chromium";v="97"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
        'sec-ch-ua-platform': '"Windows"',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Origin': 'https://services.ecourts.gov.in',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://services.ecourts.gov.in/',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie': cookie

    }
    response = s.get(link, headers=headers)
    return (response)

def process(soup, cino):
    deets = get_all(soup)
    if len(deets[1])<1:
        return 0
    disposed_date = " "
    disposed_reason = " "
    disposed_year = " "
    filing_number= deets[0][6][15:25].split('/')[0]
    filing_date = deets[0][8][-10:]
    filing_year = filing_date[-4:]
    registration_number = deets[0][9][21:31].split('/')[0]
    registration_date = deets[0][11][19:]
    registration_year = registration_date[-4:]
    stage = deets[0][16].split(": ")[-1].strip()     
    stage = re.sub(r'[^a-zA-Z ]', '',stage)  
    
    if 'CASE DISPOSED' in stage.upper():
        disposed_date = deets[0][15].split(":")[-1].strip()
        disposed_year = disposed_date[-4:]
        disposed_reason =deets[0][17].split(":")[-1].strip()
        
        judge = deets[0][18].split(':')[-1].strip()[4:].title()
        bench = deets[0][19].split(':')[-1].strip().title()
    else:
        judge = deets[0][17].split(':')[-1].strip()[4:].title()
        bench = deets[0][18].split(':')[-1].strip().title()
    if bench == '':
        bench = ' '
    statute = deets[1][0]
    provision = deets[1][1]
    if statute == 'History of Case Hearing':
        statute = " "
        provision = " "
    
    try:
        first_hearing = deets[0][14].split(": ")[1]
    except IndexError:
        first_hearing = ""

    category, sub_category, sub_sub_category, *_ = get_category_details(deets[-1])

    
    party_subview = [x for x in deets[0] if x[:2]=='1)']
    #print(party_subview)
    p_advocate, p_length, p1, p2, p3, p4, *_ = clean_parties(party_subview[0])
    r_advocate, r_length, r1, r2, r3, r4, *_ = clean_parties(party_subview[1])
    

    try:
        last_hearing = soup.select("body > form > div:nth-child(2) > table.history_table > tr > td")
        last_hearing_date = last_hearing[-3].get_text()
        last_hearing_matter = last_hearing[-1].get_text()
    except IndexError:
        last_hearing_date = ""
        last_hearing_matter = ""
    
    try:
        order_link = soup.select("#secondpage > div:nth-child(13) > table.order_table > tbody > tr:nth-child(2) > td:nth-child(4) > a")['href']
    except:
        order_link = " "

    rows = [p1,r1, cino, filing_number, filing_year, filing_date, registration_year, registration_date, stage, disposed_date, disposed_year, disposed_reason, order_link,]

    return rows

def print_all(soup):
    import unicodedata
    f = open('log.txt', 'a')
    normal = lambda x: unicodedata.normalize("NFKD", x.get_text().strip())
    f.write(f"Record: {current_record}: ")
    f.write(str([normal(x) for x in soup.find_all('span')]))
    f.write('td\n')
    f.write(str([normal(x) for x in soup.find_all('td')]))
    f.write('tr\n')
    f.write(str([normal(x) for x in soup.find_all('tr')]))
    f.write('\n')
    last_hearing = soup.select("body > form > div:nth-child(2) > table.history_table > tr > td")
    f.write(str(last_hearing))
    f.write('\n\n')

    f.close()
def get_all(soup):
    import unicodedata
    normal = lambda x: unicodedata.normalize("NFKD", x.get_text().strip())
    return (([normal(x) for x in soup.find_all('span')]), [normal(x) for x in soup.find_all('td')], [normal(x) for x in soup.find_all('tr')])\



def get_category_details(items):
    temp_category = [x.split('\n')[-1] for x in items if "Category\n" in x]
    while len(temp_category)<3:
        temp_category.append(" ")
    return temp_category

def clean_parties(parties):
    parties = re.split(r'\d+\)', parties)
    clean_parties = []
    advocates = []
    for ind, party in enumerate(parties):
        if "Advocate" in party:
            advocate = party.split("Advocate")[-1]
            advocate = re.sub(r'[^a-zA-Z ]', '',advocate).strip()
            if "FOR R" in advocate:
                advocate = advocate.split("FOR")[0]
            #print(advocate)
            advocates.append(advocate)
        
        temp_party = party.split("Advocate")[0].strip()
        if temp_party:
            temp_party = re.sub(r'[^a-zA-Z \\ \)\(]', '',temp_party)
            clean_parties.append(temp_party)
           
    if not advocates:
        advocate = " "
    else:
        advocate = advocates[0]

    clean_parties.insert(0, len(clean_parties))
    clean_parties.insert(0, advocate)
    
    while len(clean_parties) < 6:
        clean_parties.append(" ")

    return clean_parties



In [ ]:
import glob, csv
processed_cases = []
files = glob.glob("output/*.csv")
if glob.glob(files[0]):
    with open(files[0], 'r') as f:
        csv_reader = csv.reader(f)
        x = list(csv_reader)
        processed_cases.extend([c[2] for c in x])
print(processed_cases)

In [75]:
all_files = glob.glob("*.txt")
all_cases = []
for file in all_files:
    with open(file, 'r') as f:
        if '_d' in file:
            disp="disposed"
        else:
            disp="pending"

        x = json.loads(f.read())
        for cases, court_code, court in zip(x['con'], x['court_code'], x['courtNameArr']):
            if(cases and court_code and court):
                all_cases.append({
                'court_no': court_code,
                'cases': json.loads(cases),
                'court': court,
                'state_code': x['stateCode'],
                'dist_code': x['distCode'],
                'disp': disp
            })

In [76]:
for set_no in range(len(all_cases)):
    court_code = all_cases[current_set]['court_no']
    cases = all_cases[current_set]['cases']
    court = all_cases[current_set]['court']
    state_code = all_cases[current_set]['state_code']
    dist_code = all_cases[current_set]['dist_code']
    disp = all_cases[current_set]['disp']
    print(f"Starting {court}, {len(cases)} pending, {fails} failed so far. Set {current_set} of {len(all_cases)}")

    filename = f"output/{court}_{disp}.csv"
    csv_file = open(filename,'w', encoding='utf-8', newline='')
    csvwriter = csv.writer(csv_file)
    if current_record == 0:
        fields = ['P1','R1', "CINO", 'Filing Number', 'Filing Year', 'Filing Date', 'Registration Year','Registration Date', 'Stage','Disposed Date', 'Disposed Year', 'Disposal Reason', 'Doc 1 Name', 'Doc 1 Link', 'Doc 2 Name', 'Doc 2 Link', 'Court']
        csvwriter.writerow(fields) 
    
    for x in range(current_record, len(cases)):
        print(f"Processing case {current_record}", end='\r')
        cino = cases[x]['cino']
        case_no = cases[x]['case_no']
        #print(current_record, cino, case_no)
        for i in range(5):
            try:
                page, downloaded = get_case_deets(cino, case_no, court_code, state_code, dist_code)
                soup = BeautifulSoup(page)
                break
            except Exception as e:
                time.sleep(0.5+i)
                print(e, "Retrying")
        else:
            current_record+=1
            fails+=1  
            continue

        
        try:
            row = process(soup, cino)
            
            orders = soup.select("table.order_table tr")[1:]
            if len(orders) == 0:
                orders = []
            elif len(orders)<2:
                orders = [orders[-1]]
            else:
                orders = orders[-2:]
            
            for index, order in enumerate(orders):
                details = [x.get_text().strip() for x in order.select('td')]
                if not details:
                    continue
                date = details[-2]
                name = details[-1]
                query = order.select('a')[0]['href'].replace("  ", '%20')
                query = query.replace(" ", "%20")
           
                link = f"https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/{query}"
                
                order_name = f"output/orders/{row[3]} of {row[4]}_{name.title()}_{cino}_{index}.pdf"
                doc = get_case_document(link)
                if doc.headers['Content-Type'] == 'application/pdf':
                    with open(order_name, 'wb') as f:      
                        f.write(doc.content)
                        file_name = f'=hyperlink("{order_name}")'
                        row.extend([name, file_name])
                        
                else:
                    print("No file", doc.headers['Content-Type'])
        except Exception as e:      
            fails+=1    

        if row != 0:
            row.append(court)
            csvwriter.writerow(row)
                
        current_record+=1
    print(f"Processed {current_record} cases")
    csv_file.close()
    current_record = 0
    current_set = current_set+1
    

Starting District and Session Judge, South-West DWK, 37 pending, 0 failed so far. Set 0 of 37
Processed 37 cases
Starting Chief Metropolitan Magistrate, South-West DWK, 3141 pending, 0 failed so far. Set 1 of 37
No file text/html; charset=UTF-8
No file text/html; charset=UTF-8
No file text/html; charset=UTF-8


KeyboardInterrupt: 

In [56]:
print("hello", end='\r')
print("There")


There


In [77]:
date = orders[0].select('td')[1].get_text().strip()
name = orders[0].select('td')[2].get_text().strip()
link = f"https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/{quote(orders[0].select('a')[0]['href'])}"
doc = get_case_document(link)

IndexError: list index out of range

In [24]:
csv_file.close()

In [40]:
s = requests.Session()
s.get("https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=7#")

def get_case_deets(cino, case_no):
    if exists(f"delhi_case_raw/{current_record}.html"):
      with open(f"delhi_case_raw/{current_record}.html", 'r',encoding='utf-8') as f:
          #print('Existing reads')
          return (f.read(), 1)
    
    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="97", "Chromium";v="97"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
        'sec-ch-ua-platform': '"Windows"',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Origin': 'https://services.ecourts.gov.in',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://services.ecourts.gov.in/',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie': 'PHPSESSID=0s879ntfshbptb80b0hsfrj549'
    }

    data = {
      '__csrf_magic': 'sid:22e25a4a3b8a2c3dd3a08f8c43e207354a236ed2,1641670946',
      'state_code': '26',
      'dist_code': '7',
      'cino': cino,
      'case_no': case_no,
      'appFlag': '',
      'court_code': '1'
    }

    response = s.post('https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/o_civil_case_history.php', headers=headers,data=data)
    return (response.content, 0)

def get_case_document(link):

    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="97", "Chromium";v="97"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
        'sec-ch-ua-platform': '"Windows"',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Origin': 'https://services.ecourts.gov.in',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://services.ecourts.gov.in/',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie': 'PHPSESSID=0s879ntfshbptb80b0hsfrj549'

    }
    response = s.get(link, headers=headers)
    return (response)

In [71]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import os
    #initialize browser
options = webdriver.ChromeOptions()
#options.add_argument('--disable-software-rasterizer')
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("disable-infobars")
options.add_argument('ignore-certificate-errors')
options.add_argument("--log-level=3")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36")
options.add_experimental_option('prefs', {
    "download.default_directory": r"C:\Users\darsa\Desktop\DS\khc\bombay_case_files", #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    # "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
# options.add_argument("--headless")

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)

In [75]:
#driver.get("https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=1&dist_cd=23#")
driver.get("https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/display_pdf.php?filename=/orders/2019/200228002212019_1.pdf&caseno=Police%20cases%20PW/2800221/2019&cCode=1&appFlag=&normal_v=1")

In [157]:
html, field = get_case_deets("DLND010114952017", "202000002092017")
soup = BeautifulSoup(html.content)
row = process(soup)

orders = soup.select("table.order_table tr")[1:]
order = orders[-1]
date = order.select('td')[1].get_text().strip()
name = order.select('td')[2].get_text().strip()
link = f"https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/{orders[0].select('a')[0]['href']}"
doc = get_case_document(link)
if doc.headers['Content-Type'] == 'application/pdf':
    with open(f"delhi_case_files/{row[12]} of {row[13]}_{name.title()}.pdf", 'wb') as f:      
        f.write(doc.content)
        file_name = realpath(f.name)
        print(file_name)


{'Connection': 'keep-alive', 'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="97", "Chromium";v="97"', 'DNT': '1', 'sec-ch-ua-mobile': '?0', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55', 'sec-ch-ua-platform': '"Windows"', 'Content-Type': 'application/x-www-form-urlencoded', 'Accept': '*/*', 'Origin': 'https://services.ecourts.gov.in', 'Sec-Fetch-Site': 'same-origin', 'Sec-Fetch-Mode': 'cors', 'Sec-Fetch-Dest': 'empty', 'Referer': 'https://services.ecourts.gov.in/', 'Accept-Language': 'en-US,en;q=0.9', 'Cookie': 'PHPSESSID=0s879ntfshbptb80b0hsfrj549'}
C:\Users\darsa\Desktop\DS\khc\delhi_case_files\2410 of 2017_Copy Of Judgment.pdf


In [140]:
s = requests.Session()
s.get("https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=26&dist_cd=7#").headers
#s.get("https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/display_pdf.php?filename=/orders/2016/202200089262016_2.pdf&caseno=Cr%20Rev/8926/2016&cCode=1&appFlag=&normal_v=1").headers

{'Date': 'Thu, 27 Oct 2022 10:14:02 GMT', 'Server': '', 'Referrer-Policy': 'strict-origin', 'Expect-CT': 'max-age=3600, enforce', 'X-XSS-Protection': '1; mode=block', 'x-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'Set-Cookie': 'PHPSESSID=fbagu2b80d4r4q1e21124lqgqc; path=/; domain=.services.ecourts.gov.in; secure; HttpOnly; SameSite=Strict', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate', 'Pragma': 'no-cache', 'Vary': 'Accept-Encoding,User-Agent', 'content_security_policy': "script-src 'self'; object-src 'self'; img-src 'self'", 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=utf-8', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains'}

In [91]:
orders = soup.select("table.order_table tr")[1:]
order = orders[-1]
date = order.select('td')[1].get_text().strip()
doc = order.select('td')[2].get_text().strip()
link = f"https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/{orders[0].select('a')[0]['href']}"
[date, doc, link]

requests.get("https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/display_pdf.php?filename=/orders/2019/202300003722019_5.pdf&caseno=SC/372/2019&cCode=1&appFlag=&normal_v=1").content

b"\r\n<h2><table border='0' align='center'><td align='center' style='font-color:red;'> Orders is not uploaded for case number SC/372/2019</td></h2>"

In [97]:
browser.get("https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/display_pdf.php?filename=/orders/2019/202300003722019_5.pdf&caseno=SC/372/2019&cCode=1&appFlag=&normal_v=1")

In [143]:
print([].append(""))

None


In [85]:

last_hearing = soup.select("body > form > div:nth-child(2) > table.history_table > tr > td")

last_hearing_date = last_hearing[-3].get_text()
last_hearing_matter = last_hearing[-1].get_text()

In [3]:
import pandas as pd

df = pd.read_csv("2021records_new.csv")

C:\Users\darsa\AppData\Local\Temp\ipykernel_18764\2132801124.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("2021records_new.csv")


In [9]:
state_filter_list = ['state', 'Karn', 'tahsildar', 'thasildar', 'registrar', 'comm', 'chief', 'director', 'land', 'union', 'inspector', 'police', 'secretary', 'chief', 'engineer']

Index(['P1', 'P2', 'P3', 'P4', 'P_Total', 'P_Advocate', 'R1', 'R2', 'R3', 'R4',
       'R_Total', 'Filing Number', 'filing_year', 'filing_date',
       'registration_number', 'registration_year', 'registration_date', 'cnr',
       'judge', 'bench', 'statute', 'provision', 'first_hearing', 'stage',
       'category', 'sub_category', 'sub_sub_category', 'last_hearing_date',
       'date_dismissed', 'dismissal_type'],
      dtype='object')

In [14]:
df.groupby(['sub_category', 'sub_sub_category'])['P1'].count()

sub_category                                       sub_sub_category                               
                                                                                                      144
APMC-Agrl.Produce (Marketing)Regulation Act ( 2 )                                                      39
BDA-Bangalore Development Authority Act ( 3 )                                                         199
                                                   PIL-Public Interest Litigation ( 1 )                 3
C-Cinema Regulation Act ( 4 )                                                                           3
                                                                                                     ... 
T-Tax Matter ( 21 )                                RES-Residuary ( 112 )                              241
                                                   TAR-Central Excise/Customs Act (Tariff) ( 123 )      2
T-Tax Matter ( 32 )                                IT

In [17]:
import glob
files = glob.glob("case_files/2018*")
count = 0
for fil in files:
    with open(fil, 'r') as f:
        a = f.read()
        if "403" in a:
            count+=1
print(count)
fin()

16319


In [16]:
import winsound
def fin():
    duration = 250  # milliseconds
    freq = 740  # Hz
    for i in range(3):
        winsound.Beep(freq, duration)

In [3]:
import cv2
import pytesseract
import numpy as np
import string
import re

def detect_captcha(image):
    pytesseract.pytesseract.terreract_cmd = {
        r'/usr/bin/tesseract'
    }
    src = cv2.imdecode(np.frombuffer(image.read(), np.uint8), 1)
    ret,img = cv2.threshold(np.array(src), 125, 255, cv2.THRESH_BINARY)
    s = pytesseract.image_to_string(img)
    output_txt = re.sub(r'[^a-zA-Z0-9]','',s)
    return output_txt

In [19]:
import requests

cookies = {
    'PHPSESSID': '3hgqhdc1u5k1a5sl2nrf2doc3c',
}

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Cookie': 'PHPSESSID=3hgqhdc1u5k1a5sl2nrf2doc3c',
    'Origin': 'https://services.ecourts.gov.in',
    'Referer': 'https://services.ecourts.gov.in/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

data = {
    'court_codeArr': '17',
    'state_code': '1',
    'dist_code': '21',
    'search_act': '',
    'actcode': '215',
    'f': 'Pending',
    'under_sec': '',
    'action_code': 'showRecords',
    'captcha': 'wd57fy',
    'lang': '',
}

response = requests.post(
    'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise_qry.php',
    cookies=cookies,
    headers=headers,
    data=data,
)
response.content

b'{"con":"Invalid Captcha"}'

In [2]:
import requests

cookies = {
    'PHPSESSID': '3hgqhdc1u5k1a5sl2nrf2doc3c',
}




'/ecourtindia_v4_bilingual/securimage/securimage_show.php?a5fe4107d8660bff0a75efd489c252ea'

In [6]:
import io
from bs4 import BeautifulSoup
from twocaptcha import TwoCaptcha
import json
import requests
import re

def get_court_complex_cases(link, section=379):
    #Get captcha and site
    print("Starting ", link)
    headers = {
    'Accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Referer': 'https://services.ecourts.gov.in/',
    'Sec-Fetch-Dest': 'image',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}
    s = requests.session()
    resp = s.get(link, headers=headers, timeout=10)

    state_code, dist_code = re.findall('=(\d{1,2})', link)
    soup = BeautifulSoup(resp.content)
    court_codes = [x.attrs['value'].split("@")[1] for x in soup.find(id='court_complex_code').find_all('option')[1:]]
    print(court_codes)
    
    for court in court_codes:
        ActNodata = {
            'court_codeArr': court,
            'state_code': state_code,
            'dist_code': dist_code,
            'search_act': '',
            'action_code': 'fillActType',

        }
        response = s.post(
            'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise_qry.php',
            headers=headers,
            data=ActNodata,
            timeout=5
        )
        codes = str(response.content).split("#")
        actCode = 0
        for code in codes:
            if "ipc" in code.lower() or "indian penal" in code.lower():
                actCode = code.split("~")[0]
                print(actCode, code)

        for case_type in ['Pending', 'Disposed']:

            solver = TwoCaptcha('3d5411601bd36ac99dab2eeade90f405')
            for x in range(3):
                try:
                    print(f"Trying {court} and {case_type}")
                    captcha_link = "https://services.ecourts.gov.in"+soup.find(id='captcha_image').attrs['src']
                    with open('image.png', 'wb') as f:
                        f.write(s.get(captcha_link, headers=headers, timeout=10).content)
                    captcha = solver.normal('image.png')
                    data = {
                        'court_codeArr': court,
                        'state_code': state_code,
                        'dist_code': dist_code,
                        'search_act': '',
                        'actcode': actCode,
                        'f': case_type,
                        'under_sec': section,
                        'action_code': 'showRecords',
                        'captcha': captcha['code'],
                        'lang': '',
                    }
                    print(f"Data is {data}")
                    response = s.post(
                        'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise_qry.php',
                        headers=headers,
                        data=data,
                        timeout=5
                    )
                    
                    data = json.loads(response.content)
                    if data['con']=='Invalid Captcha':
                        "INVALID"
                        continue
                    else: 
                        break
                except Exception as e:
                    print(e)
                    continue

            data['stateCode'] = state_code
            data['distCode'] = dist_code
            with open(f"output/case_list/{court}_{state_code}_{dist_code}_{case_type}.txt", 'w') as f:
                f.write(json.dumps(data))


In [7]:
links = ["https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=1&dist_cd=24"]
try:
    get_court_complex_cases(links[0])
except requests.Timeout:
    print("TIMEOUT")


Starting  https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise.php?state=D&state_cd=1&dist_cd=24
['6', '15', '8', '10', '4', '11', '9', '16', '12,13', '5', '7,14', '1,2,3', '17']
1 1~INDIAN PENAL CODE
Trying 6 and Pending
<class 'Exception'>
Trying 6 and Pending
<class 'Exception'>
Trying 6 and Pending
<class 'Exception'>


UnboundLocalError: local variable 'data' referenced before assignment

In [14]:
soup = BeautifulSoup(resp.content)
[x.attrs['value'].split("@")[1] for x in soup.find(id='court_complex_code').find_all('option') if len(x)>1]

NameError: name 'resp' is not defined

In [11]:
data = {
    'court_codeArr': '17',
    'state_code': '1',
    'dist_code': '21',
    'search_act': '',
    'actcode': '1',
    'f': 'Pending',
    'under_sec': '379',
    'action_code': 'fillActType',

}

response = s.post(
    'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/s_actwise_qry.php',
    headers=headers,
    data=data,
)
codes = str(response.content).split("#")
for code in codes:
    if "ipc" in code.lower() or "indian penal" in code.lower():
        print(code.split("~")[0])

1


In [7]:
import glob
all_files = glob.glob("output/case_list/*.txt")
all_cases = []
for file in all_files:
    with open(file, 'r') as f:
        if '_Disp' in file:
            disp="disposed"
        else:
            disp="pending"

        x = json.loads(f.read())
        for cases, court_code, court in zip(x['con'], x['court_code'], x['courtNameArr']):
            if(cases and court_code and court):
                all_cases.append({
                'court_no': court_code,
                'cases': json.loads(cases),
                'court': court,
                'state_code': x['stateCode'],
                'dist_code': x['distCode'],
                'disp': disp
            })
count = 0
for cases in all_cases:
    count+=len(cases['cases'])

print(f"Processing {count} cases total")

Processing 124264 cases total
